# Comparison of Dipoles across EM Solvers
This notebook is used to compare dipoles simulated three different ways: using an old version of ParaProp, a new version of ParaProp, and Meep (finite difference time domain). It's purpose is to test if the new version of ParaProp, with improved approximations, works better than the old version, i.e. is closer to finite difference time domain solutions 

#### Notes
* The simulations of the ParaProp versions are done in this code
* The simulations using Meep are done in $\verb"MeepDipoleSimulation.py"$. The data is saved to npy files and opened here.
* To adjust the various improvements to ParaProp you have to adjust that in $\verb"paraPropPythonNew.py"$

#### Revision History
* Last revised 10-May-2022 by Leo Deer (deer.279@osu.edu).

In [ ]:
##### importing necessary packages #####

%matplotlib inline
import paraPropPython as ppp      # This is the original version of ParaProp
import paraPropPythonNew as pppn     # This is the new verion of ParaProp
import numpy as np
import matplotlib.pyplot as plt
import util as util

### Setting up the ParaProp simulations

In [ ]:
##### setting up parameters and conditions of ParaProp simulations #####

### first, initialize the dimensions of the simulation ###
iceDepth = 75. #depth below the surface of the ice (m)
iceLength = 100. #radial length away from the antenna (m)
dx = 0.08 # grid size in the radial direction (m)
dz = 0.08 # grid size in the vertical direction (m)

### next, initialize the ParaProp Dipole specs ###
freq = 0.1 # frequency in GHz
sourceDepth = 9.9 # depth of the top dipole of the array (m)
deltasource = False # If true, dipole source is like a delta function. If false, dipole source has a sin^2 pattern.

### next, initialize the simulation ###
sim = ppp.paraProp(iceLength, iceDepth, dx, dz, refDepth=sourceDepth)
simn = pppn.paraProp(iceLength, iceDepth, dx, dz, refDepth=sourceDepth)

### useful arrays for plottinng ###
z = sim.get_z()
x = sim.get_x()

zn = simn.get_z()
xn = simn.get_x()

In [ ]:
### next, initialize the index of refraction n ###

## an example of defining n and its first two derivatives as a function of z ## 
def southpole(z):
    A=1.78
    B=-0.43
    C=-0.0132
    return A+B*np.exp(C*z)

def southpole(zn):
    A=complex(1.78,0)
    B=complex(-0.43,0)
    C=complex(-0.0132,0)
    return A+B*np.exp(C*zn)

def southpole_p(zn):
    A=1.78
    B=-0.43
    C=-0.0132
    return C*B*np.exp(C*zn)

def southpole_p_p(zn):
    A=1.78
    B=-0.43
    C=-0.0132
    return C*C*B*np.exp(C*zn)

sim.set_n(nFunc=southpole)
simn.set_n(nFunc=southpole, nprFunc=southpole_p, nprprFunc=southpole_p_p)

## plot of n ##
plt.plot(z, simn.get_n()[:,0], color='black')
plt.ylabel('n')
plt.xlabel('z (m)')
plt.title('Index of Refraction Profile of Simulation')
plt.xlim(z[0], z[-1])

plt.show()

In [ ]:
### defining the sources ###

### first we define the delta funciton source with a vector ###
dipoleDepth = float(sourceDepth) 

arraySize = round(iceDepth - sourceDepth)
sourceArray = []
for i in range(int(round(arraySize/dz))):
    sourceArray.append(0)
sourceArray[int(round(dipoleDepth)/dz-1)] = 1 
sourceArray[int(round(dipoleDepth)/dz-2)] = 1 

### now we define the sources ###
if deltasource == True:
    sim.set_user_source_profile('vector', z0=0, sVec=sourceArray, sFunc=None)
    simn.set_user_source_profile('vector', z0=0, sVec=sourceArray, sFunc=None)
else:
    sim.set_dipole_source_profile(freq, sourceDepth)
    simn.set_dipole_source_profile(freq, sourceDepth)

### set a cw signal ###
sim.set_cw_source_signal(freq)
simn.set_cw_source_signal(freq)    
    
## plot the source profile ##
plt.plot(z, abs(sim.get_source_profile()), color='black')
plt.ylabel('Field')
plt.xlabel('z (m)')
plt.title('Source Profile of Simulation')
plt.xlim(sourceDepth - 1, sourceDepth + 1)
plt.show()

In [ ]:
### run the solver ###
sim.do_solver()
simn.do_solver()

#### relevant quantities ####

PP_Dipole = abs(sim.get_field())  # Old version ParaProp field
PP_Dipolen = abs(simn.get_field())  # Old version ParaProp field
absdiffPP = abs(sim.get_field()-simn.get_field())  # Absolute value of the difference in ParaProp fields
diffabsPP = abs(sim.get_field())-abs(simn.get_field())  # Difference in the absolute value of the ParaProp fields

### Plotting the Fields

In [ ]:
### plot the absolute value of field for the old ParaProp dipole ###
fig = plt.figure()
ax = fig.add_subplot(111)

plt.imshow(np.transpose(PP_Dipole), aspect='auto', cmap='hot',  vmin=1e-6, vmax=5e-3, 
          extent=(xn[0], xn[-1], zn[-1], zn[0]))
plt.colorbar()
plt.title("Old ParaProp Absolute Field, " + str(int(freq.real*1000))+" MHz")
plt.xlabel("x (m)")
plt.ylabel("z (m)")
plt.show()


### plot the absolute value of field for the new ParaProp dipole ###
fig = plt.figure()
ax = fig.add_subplot(111)

plt.imshow(np.transpose(PP_Dipolen), aspect='auto', cmap='hot',  vmin=1e-6, vmax=5e-3, 
          extent=(x[0], x[-1], z[-1], z[0]))
plt.colorbar()
plt.title("New ParaProp Absolute Field, " + str(int(freq*1000))+" MHz")
plt.xlabel("x (m)")
plt.ylabel("z (m)")
plt.show()


### plot the absolute value of the difference in ParaProp fields ###
fig = plt.figure()
ax = fig.add_subplot(111)

plt.imshow(np.transpose(absdiffPP), aspect='auto', cmap='hot',  vmin=1e-6, vmax=5e-3, 
          extent=(xn[0], xn[-1], zn[-1], zn[0]))
plt.colorbar()
plt.title("Absolute value of the difference in ParaProp fields")
plt.xlabel("x (m)")
plt.ylabel("z (m)")
plt.show()


### plot the difference in the absolute value of the ParaProp fields ###
fig = plt.figure()
ax = fig.add_subplot(111)

plt.imshow(np.transpose(diffabsPP), aspect='auto', cmap='hot',  vmin=1e-6, vmax=3e-3, 
          extent=(xn[0], xn[-1], zn[-1], zn[0]))
plt.colorbar()
plt.title("Difference in the absolute value of the ParaProp fields")
plt.xlabel("x (m)")
plt.ylabel("z (m)")
plt.show()

### Grabbing the dipole field from the Meep simulation

In [ ]:
# the array is saved in the file geekfile.npy 
Meep_Dipole = np.load('MeepDipoleData.npy')

### Plotting the Meep dipole

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

plt.imshow(Meep_Dipole, aspect='auto', cmap='hot',  vmin=1e-6, vmax=5e-4, 
          extent=(x[0], x[-1], z[-1], z[0]))
plt.colorbar()
plt.title("Meep Absolute Field zoomed in, " + str(int(freq*1000))+" MHz")
plt.xlabel("x (m)")
plt.ylabel("z (m)")
#plt.ylim(12,9) # if we need to zoom in
#plt.xlim(0,3) # if we need to zoom in
plt.show()

### Comparing the Meep fields with the ParaProp fields

In [ ]:
### First we need to create and adjust some new quantites ###

## This ensures the Meep and ParaProp fields have the same dimensions ##
PP_Dipole = [[PP_Dipole[i,j] for i in range(int(len(Meep_Dipole[0])))] for j in range(int(len(Meep_Dipole)))]
PP_Dipolen = [[PP_Dipolen[i,j] for i in range(int(len(Meep_Dipole[0])))] for j in range(int(len(Meep_Dipole)))]

## New comparison arrays ##
diff = PP_Dipolen - abs(Meep_Dipole)   ## Difference in the absolute value of the new ParaProp field and Meep field

Here we look at a difference in the absolute Meep field and new ParaProp field. We zoom into this to make sure that the sources line up, otherwise this will create a greater difference than there actually is. Once we check the lineup we either go back and adjust $\verb"sourceDepth"$ or continue.

In [ ]:
fig1 = plt.figure()

plt.imshow(diff, aspect='auto', cmap='RdBu',  vmin=-5e-4, vmax=5e-4, 
          extent=(x[0], x[-1], z[-1], z[0]))
plt.colorbar()
plt.title("Difference in absolute ParaProp and Meep Fields")
plt.xlabel("x (m)")
plt.ylabel("z (m)")
plt.show()

plt.imshow(diff, aspect='auto', cmap='RdBu', vmin=-5e-4, vmax=5e-4,
          extent=(x[0], x[-1], z[-1], z[0]))
plt.colorbar()
plt.title("Difference in absolute ParaProp and Meep Fields ZOOMED")
plt.xlabel("x (m)")
plt.ylabel("z (m)")
plt.ylim(12,9) # adjust these to zoom in
plt.xlim(0,3) # adjust these to zoom in
plt.show()

### Plot slices of fields to compare

In [ ]:
### plot absolute value of fields for horizontal slice of simulation ###

z0 = 50. # depth of slice (m)
indexz = int((z0+25.)/dz)

## quantities to help with plotting ##
norm_factormp = np.sum(PP_Dipolen[indexz])/np.sum(abs(Meep_Dipole[indexz]))
norm_factorpp = float(np.sum(PP_Dipolen[indexz])/np.sum(PP_Dipole[indexz]))
scaledPP_Dipole = [norm_factorpp * x for x in PP_Dipole[indexz]]

## plot individual slices ##
plt.plot(x, PP_Dipole[indexz], color='blue', label='PP Old')
plt.xlabel('x (m)')
plt.ylabel('Field')
plt.title("Old PP Absolute field at depth " + str(z0) + " m" )
plt.show()

plt.plot(x, PP_Dipolen[indexz], color='black', label='PP New')
plt.xlabel('x (m)')
plt.ylabel('Field')
plt.title("New OO PP Absolute field at depth " + str(z0) + " m" )
plt.show()

plt.plot(x, abs(Meep_Dipole[indexz]), color='red', label='Meep')
plt.xlabel('x (m)')
plt.ylabel('Field')
plt.title("Meep Absolute field at Depth " + str(z0) + " m" )
plt.ylim(0,0.0003)
plt.show()

## normalize fields to help compare ##
norm_factormp = np.sum(PP_Dipolen[indexz])/np.sum(abs(Meep_Dipole[indexz]))
norm_factorpp = float(np.sum(PP_Dipolen[indexz])/np.sum(PP_Dipole[indexz]))
scaledPP_Dipole = [norm_factorpp * x for x in PP_Dipole[indexz]]

## compare the slices ##
plt.plot(x, norm_factormp*abs(Meep_Dipole[indexz]), color='red', label='Meep')
plt.plot(x, PP_Dipolen[indexz], color='black', label='PP New')
plt.legend()
plt.xlabel('x (m)')
plt.ylabel('Field')
plt.title("Absolute field at Depth " + str(z0) + " m" )
plt.ylim(0.,0.003)
plt.show()

#fig = plt.figure()

plt.plot(x, norm_factormp*abs(Meep_Dipole[indexz]), color='red', label='Meep')
plt.plot(x, PP_Dipolen[indexz], color='black', label='PP New')
plt.plot(x, scaledPP_Dipole, color='blue', label='PP Old')
plt.legend()
plt.xlabel('x (m)')
plt.ylabel('Field')
plt.title("Absolute field at Depth " + str(z0) + " m" )
plt.ylim(0.,0.003)
plt.show()
#fig.savefig('sinkcompZ75.png')



In [ ]:
### plot absolute value of fields for vertical slice of simulation ###

x0 = 100. # range of slice (m)
indexx = int(x0/dx)

## quantities to help with plotting ##
PP_DipoleT = np.transpose(PP_Dipole)
PP_DipoleTn = np.transpose(PP_Dipolen)
Meep_DipoleT = np.transpose(Meep_Dipole)
z = [z[i] for i in range(len(PP_DipoleT[indexx])) ]
norm_factormp = np.sum(PP_DipoleTn[indexx])/np.sum(Meep_DipoleT[indexx])
norm_factorpp = np.sum(PP_DipoleTn[indexx])/np.sum(PP_DipoleT[indexx])

## plot individual slices ##
plt.plot(PP_DipoleT[indexx], z, color='blue', label='PP Old')
plt.ylabel('z (m)')
plt.xlabel('Field')
plt.title("Old PP Absolute field at range " + str(z0) + " m" )
plt.show()

plt.plot(PP_DipoleTn[indexx], z, color='black', label='PP New')
plt.ylabel('z (m)')
plt.xlabel('Field')
plt.title("New PP Absolute field at range " + str(z0) + " m" )
plt.show()

plt.plot(Meep_DipoleT[indexx], z, color='red', label='Meep')
plt.ylabel('z (m)')
plt.xlabel('Field')
plt.title("Meep Absolute field at range " + str(z0) + " m" )
plt.show()

## compare the slices ##
plt.plot(norm_factormp*Meep_DipoleT[indexx], z, color='red', label='Meep')
plt.plot(PP_DipoleTn[indexx], z, color='black', label='PP New')
plt.legend()
plt.ylabel('z (m)')
plt.xlabel('Field')
plt.title("Absolute field at range " + str(x0) + " m" )
plt.show()

#fig = plt.figure()
plt.plot(norm_factormp*Meep_DipoleT[indexx], z, color='red', label='Meep')
plt.plot(PP_DipoleTn[indexx], z, color='black', label='PP New')
scaledPP_DipoleT = [norm_factorpp * x for x in PP_DipoleT[indexx]]
plt.plot(scaledPP_DipoleT, z, color='blue', label='PP Old')
plt.legend()
plt.ylabel('z (m)')
plt.xlabel('Field')
plt.title("Absolute field at range " + str(x0) + " m" )
plt.show()
#fig.savefig('sinkcompX100.png')